Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import sys
import os
import gc
import cv2

import csv
from google.colab import files
from google.colab import drive

from scipy.ndimage.interpolation import rotate
from scipy.ndimage.interpolation import shift
from scipy.ndimage.interpolation import zoom
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, Callback, EarlyStopping

from sklearn.model_selection import train_test_split

from keras.layers import GaussianNoise
from keras import Model
from keras import layers
from keras.layers.convolutional import Conv2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
np.set_printoptions(threshold=sys.maxsize)

np.random.seed(121)

Loading files

In [ ]:
drive.mount('/content/drive')                   #mount the file
os.chdir("/content/drive/My Drive")             #change file directory
dataset = h5py.File('MNIST_synthetic.h5', 'r')  #load file
print(dataset.keys())

Mounted at /content/drive
<KeysViewHDF5 ['test_dataset', 'train_dataset', 'train_labels']>


Extracting data

In [ ]:
X_train = dataset.get('train_dataset')
Y_train = dataset.get('train_labels')
Thex_test = dataset.get('test_dataset')

X_train = np.array(X_train)
Y_train = np.array(Y_train)
Thex_test = np.array(Thex_test)         #load file

Preprocessing, including normalization and using one-hot label.

In [ ]:
X_train = X_train.astype('float32')
Thex_test = Thex_test.astype('float32')
X_train /= 255
Thex_test /= 255
Y_train = np_utils.to_categorical(Y_train, 11)   #one hot coding

Further preprocessing, namely resize to twice (for x_train and x_test)

In [ ]:
def image_preprocessing(img, dimx = 64, dimy = 64):
    img1 = cv2.resize(img, (dimx, dimy), interpolation = cv2.INTER_LINEAR)   #resize data
    img1 = img1[..., np.newaxis]
    return img1


X_train = np.array([image_preprocessing(i,128,128) for i in X_train])     #preprocessing
Thex_test = np.array([image_preprocessing(i,128,128) for i in Thex_test])
print(X_train.shape)

(56000, 128, 128, 1)


In [ ]:
batchsize = 128    #batchsize

Split data

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=21, shuffle=True)  #split data
print(x_train.shape)
print(y_train.shape)

(50400, 128, 128, 1)
(50400, 5, 11)


Generate more image data based on x_train

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=10,         #rotation range
    shear_range=0.1,           #shear range
    zoom_range=0.1)            #zoom range

Train = datagen.flow(x_train, y_train, batch_size=batchsize, seed=20)    #generate random training data

put x_train into model.fit

In [ ]:
def data_augmentor(data_gen):
    for x, y in data_gen:
        y = [y[:, i] for i in range(5)]    #select each
        yield x, y

Define model

In [ ]:
def cnn_model(input_dim=(128, 128, 1), out_dim=11):    #input dim = 128,128, outdim = 11
    inputss = Input(shape=input_dim)

    x = BatchNormalization()(inputss)      #batch normalization
    x = Dropout(.1)(x)                     #dropout
    x = GaussianNoise(0.1)(x)              #gaussian noise
    x = Conv2D(32, (3,3), activation='relu', padding='same')(x)          #convolutional layer(size=32, activation is relu)
    x = Conv2D(32, (3,3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)  
    x = Dropout(.25)(x)

    x = BatchNormalization()(x)  
    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)          #convolutional layer(size=64, activation is relu)
    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)      
    x = Dropout(.25)(x)

    x = BatchNormalization()(x)
    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)         #convolutional layer(size=128, activation is relu)
    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)    
    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)  
    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)   
    x = MaxPooling2D(pool_size=(2,2))(x) 
    x = Dropout(.25)(x) 

    x = BatchNormalization()(x)
    x = Conv2D(256, (3,3), activation='relu', padding='same')(x)         #convolutional layer(size=256, activation is relu)
    x = Conv2D(256, (3,3), activation='relu', padding='same')(x)    
    x = Conv2D(256, (3,3), activation='relu', padding='same')(x)  
    x = Conv2D(256, (3,3), activation='relu', padding='same')(x)  
    x = MaxPooling2D(pool_size=(2,2))(x)  
    x = Dropout(.25)(x) 
    x = GaussianNoise(0.05)(x)

    x = BatchNormalization()(x)
    x = Flatten()(x)                                 #flatten 2D to vectorize
    x = Dense(512,activation='relu')(x)              #dense layer
    x = Dropout(.5)(x)                               #dropout
    x = Dense(512,activation='relu')(x)
    x = Dropout(.5)(x)
    x = GaussianNoise(0.05)(x)
    outputss = [Dense(11, activation='softmax', name=f"digit_{i}")(x) for i in range(5)]    #output softmax
    
    model = Model(inputs=inputss, outputs=outputss)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  #compile model, using cross entropy
    return model

some callbacks

In [ ]:
early_stopping = EarlyStopping(             #early stopping
	monitor = "val_loss",                     #monitor validation loss
	patience = 10,
	verbose = 1,
	restore_best_weights = True
)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, min_delta=0.00001,
                              patience=6, verbose=1)     #reduce learning rate on plateau

checkpoint_filepath = '/content/drive/My Drive/checkpoints2/weights.{epoch:02d}.h5'
model_checkpoint_callback = ModelCheckpoint(   #model check point
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    #save_best_only=True
)

training the model

In [ ]:
model = cnn_model(input_dim=(128, 128, 1))                         #define cnn model
history = model.fit(
    data_augmentor(Train),                                         #training data
    validation_data = (x_val, [y_val[:,i] for i in range(5)]),     #validtion data
    batch_size = batchsize,                                        #batchsize=128
    epochs = 60,                                                   #number of epoch
    steps_per_epoch=len(X_train)/20,                               
    callbacks = [reduce_lr, model_checkpoint_callback],            #callbacks
    verbose = 1
)

Epoch 1/60
2800/2800 [==============================] - 393s 140ms/step - loss: 5.0650 - digit_0_loss: 1.4212 - digit_1_loss: 1.2698 - digit_2_loss: 1.0561 - digit_3_loss: 0.8083 - digit_4_loss: 0.5096 - digit_0_accuracy: 0.4766 - digit_1_accuracy: 0.5302 - digit_2_accuracy: 0.6092 - digit_3_accuracy: 0.7048 - digit_4_accuracy: 0.8197 - val_loss: 0.4951 - val_digit_0_loss: 0.1050 - val_digit_1_loss: 0.1269 - val_digit_2_loss: 0.1176 - val_digit_3_loss: 0.0755 - val_digit_4_loss: 0.0701 - val_digit_0_accuracy: 0.9805 - val_digit_1_accuracy: 0.9812 - val_digit_2_accuracy: 0.9812 - val_digit_3_accuracy: 0.9886 - val_digit_4_accuracy: 0.9862
Epoch 2/60
2800/2800 [==============================] - 391s 140ms/step - loss: 0.8045 - digit_0_loss: 0.2086 - digit_1_loss: 0.1960 - digit_2_loss: 0.1643 - digit_3_loss: 0.1335 - digit_4_loss: 0.1020 - digit_0_accuracy: 0.9452 - digit_1_accuracy: 0.9455 - digit_2_accuracy: 0.9523 - digit_3_accuracy: 0.9602 - digit_4_accuracy: 0.9688 - val_loss: 0.147

convert back

In [ ]:
def onehot_to_label(arr):       #convert back
    arr = np.array(arr)
    R = np.argmax(arr, axis=2)  #select the label
    R = np.transpose(R)
    return R

write the result in csv file

In [ ]:
def write_to_csv(prediction):
    preds = onehot_to_label(prediction)                       #convert label
    predss = [[i, ''.join(str(x) for x in pred)] for i, pred in enumerate(preds)]    #form the prediction
    filename = "sample.csv"                                   #download file
    with open(filename, 'w', newline='') as csvfile:          #open file
        csvwriter = csv.writer(csvfile, delimiter=',')        #write file
        csvwriter.writerow(["Id", "Label"])                   #write the title
        csvwriter.writerows(predss)                           #write the rows(all the data)
    files.download('sample.csv')                              #download file

save model and predict

In [ ]:
#model.save_weights('weights.h5', overwrite=True)
#You may not want to run this. ------------------------------------------
model1 = cnn_model(input_dim=(128, 128, 1))
file = '/content/drive/My Drive/checkpoints2/weights.02.h5'
model1.load_weights(file)                                     #load weights

preds = model1.predict(Thex_test)                             #prediction
write_to_csv(preds)                                           #write to csv

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

plot graph

In [ ]:
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
accuracy1 = history_dict['digit_0_accuracy']
accuracy2 = history_dict['digit_1_accuracy']
accuracy3 = history_dict['digit_2_accuracy']
accuracy4 = history_dict['digit_3_accuracy']
accuracy5 = history_dict['digit_4_accuracy']
val_accuracy1 = history_dict['val_digit_0_accuracy']
val_accuracy2 = history_dict['val_digit_1_accuracy']
val_accuracy3 = history_dict['val_digit_2_accuracy']
val_accuracy4 = history_dict['val_digit_3_accuracy']
val_accuracy5 = history_dict['val_digit_4_accuracy']
 
epochs = range(1, len(loss_values) + 1)
fig, ax = plt.subplots(1, 3, figsize=(18, 5))
#
# Plot the model accuracy vs Epochs
#
ax[0].plot(epochs, accuracy1, 'r', label='Training accuracy1')
ax[0].plot(epochs, accuracy2, 'y', label='Training accuracy2')
ax[0].plot(epochs, accuracy3, 'b', label='Training accuracy3')
ax[0].plot(epochs, accuracy4, 'g', label='Training accuracy4')
ax[0].plot(epochs, accuracy5, 'c', label='Training accuracy5')

ax[0].set_title('Training Accuracy', fontsize=16)
ax[0].set_xlabel('Epochs', fontsize=16)
ax[0].set_ylabel('Accuracy', fontsize=16)
ax[0].legend()


ax[1].plot(epochs, val_accuracy1, 'r', label='Validaton accuracy1')
ax[1].plot(epochs, val_accuracy2, 'y', label='Validaton accuracy2')
ax[1].plot(epochs, val_accuracy3, 'b', label='Validaton accuracy3')
ax[1].plot(epochs, val_accuracy4, 'g', label='Validaton accuracy4')
ax[1].plot(epochs, val_accuracy5, 'c', label='Validaton accuracy5')
ax[1].set_title('Validation Accuracy', fontsize=16)
ax[1].set_xlabel('Epochs', fontsize=16)
ax[1].set_ylabel('Accuracy', fontsize=16)
ax[1].legend()

#
# Plot the loss vs Epochs
#
ax[2].plot(epochs, loss_values, 'b', label='Training loss') 
ax[2].plot(epochs, val_loss_values, 'r', label='Validation loss')
ax[2].set_title('Training & Validation Cross-Entropy Loss', fontsize=16)
ax[2].set_xlabel('Epochs', fontsize=16)
ax[2].set_ylabel('Loss', fontsize=16)
ax[2].legend()

fig.show()

"history_dict = history.history\nloss_values = history_dict['loss']\nval_loss_values = history_dict['val_loss']\naccuracy1 = history_dict['digit_0_accuracy']\naccuracy2 = history_dict['digit_1_accuracy']\naccuracy3 = history_dict['digit_2_accuracy']\naccuracy4 = history_dict['digit_3_accuracy']\naccuracy5 = history_dict['digit_4_accuracy']\nval_accuracy1 = history_dict['val_digit_0_accuracy']\nval_accuracy2 = history_dict['val_digit_1_accuracy']\nval_accuracy3 = history_dict['val_digit_2_accuracy']\nval_accuracy4 = history_dict['val_digit_3_accuracy']\nval_accuracy5 = history_dict['val_digit_4_accuracy']\n \nepochs = range(1, len(loss_values) + 1)\nfig, ax = plt.subplots(1, 2, figsize=(14, 6))\n#\n# Plot the model accuracy vs Epochs\n#\nax[0].plot(epochs, accuracy1, 'r', label='Training accuracy1')\nax[0].plot(epochs, accuracy2, 'y', label='Training accuracy2')\nax[0].plot(epochs, accuracy3, 'b', label='Training accuracy3')\nax[0].plot(epochs, accuracy4, 'g', label='Training accuracy4'